---
title: "Winners and Losers"
author: "spulick"
format:
  html:
    code-fold: true
    code-summary: "Show the code"
  docx:
    order: 1
    reference-doc: meta/custom-reference-doc.docx
    toc-depth: 1
    fig-align: "center"
    fig-dpi: 600
    df-print: default
    filters:
      - quarto
      - meta/pagebreak.lua
    execute:
      echo: false
      output: true
---

In [10]:
%%html
<style>
/* Any CSS style can go in here. */
.dataframe th {
    font-size: 10pt;
}
.dataframe td {
    font-size: 8pt;
}
</style>

This document explores the expected changes to the FA Grant General Purpose component entitlements. 

> The distribution of change across percentage categories only looks entitilements in the 2026-27 financial year. 

> Future entitlements have been discounted at $r=2.50\%$ when computing total expected gains and losses over the four year period.

\newpage

In [12]:
import pandas as pd
from fagrants_module import fagrants_gccsa as fagrants

def weighted_group_delta(grouped_df):
    return_dict = {}

    for year in range(2025, 2029):
        total_weight = grouped_df[f"ERP_{year}"].mean()

        return_dict[f"Population_{year}"] = total_weight

        weighted_mean = (
            grouped_df[f"Grant Delta_{year}"] * grouped_df[f"ERP_{year}"]
        ).sum() / grouped_df[f"ERP_{year}"].sum()

        return_dict[f"Grant Delta_{year}"] = weighted_mean

        return_dict[f"Grant Delta %_{year}"] = (
            grouped_df[f"Grant Delta %_{year}"] * grouped_df[f"ERP_{year}"]
        ).sum() / grouped_df[f"ERP_{year}"].sum()

    return pd.Series(return_dict)

def grants_simulation(data, pc_min, pc_min_base):
    inst = fagrants.fagrants_model(
    data_path=data,
    per_capita_minimum=pc_min,
    per_capita_minimum_base=pc_min_base,
    )

    sim_grants = inst.run()

    sim_grants["Sim_i"] = sim_grants["UID"].str.split("_").str[1].astype(int)

    for year in range(2025, 2029):
        sim_grants[f"Grant Delta_{year}"] = (
            sim_grants[f"Grant_{year}"] - sim_grants[f"Grant_base_{year}"]
        )

        sim_grants[f"Grant Delta %_{year}"] = (
            sim_grants[f"Grant Delta_{year}"] * 100 / sim_grants[f"Grant_base_{year}"]
        )

    return sim_grants

def grants_delta(sim_grants):
    sim_grants_grouped = (
        sim_grants.groupby(["Jurisdiction", "ACLG", "LGA"])
        .apply(weighted_group_delta, include_groups=False)
        .reset_index()
    )

    pop_melted = pd.melt(
        sim_grants_grouped,
        id_vars=["Jurisdiction","ACLG", "LGA"],
        value_vars=[f"Population_{year}" for year in range(2025, 2029)],
        var_name="Year",
        value_name="Population",
    )

    sim_grants_melted = pd.melt(
        sim_grants_grouped,
        id_vars=["Jurisdiction","ACLG", "LGA"],
        value_vars=[f"Grant Delta_{year}" for year in range(2025, 2029)],
        var_name="Year",
        value_name="Expected Change in GPG ($)",
    )

    for year in range(2025, 2029):
        sim_grants_grouped[f"Grant Delta Per Capita_{year}"] = sim_grants_grouped[f"Grant Delta_{year}"]/ sim_grants_grouped[f"Population_{year}"]

    sim_grants_melted_per_capita = pd.melt(
        sim_grants_grouped,
        id_vars=["Jurisdiction", "ACLG", "LGA"],
        value_vars=[f"Grant Delta Per Capita_{year}" for year in range(2025, 2029)],
        var_name="Year",
        value_name="Expected Change in GPG Per Capita ($)",
    )

    sim_grants_melted_per_cent = pd.melt(
        sim_grants_grouped,
        id_vars=["Jurisdiction", "ACLG", "LGA"],
        value_vars=[f"Grant Delta %_{year}" for year in range(2025, 2029)],
        var_name="Year",
        value_name="Expected Change in GPG (%)",
    )

    sim_grants_melted["Year"] = sim_grants_melted["Year"].str.split("_").str[1].astype(int)

    sim_grants_melted["Expected Change in GPG (%)"] = sim_grants_melted_per_cent["Expected Change in GPG (%)"]

    sim_grants_melted["Expected Change in GPG Per Capita ($)"] = sim_grants_melted_per_capita["Expected Change in GPG Per Capita ($)"]

    sim_grants_melted["Expected Change in GPG (%) Bin"] = pd.cut(sim_grants_melted["Expected Change in GPG (%)"], bins=[-float("inf"), -40, -20, -10, -5 , 0, 5, 10, 20, 40, float("inf")], labels=["Decreased more than 40%", "Decreased 20-40%", "Decreased 10-20%", "Decreased 5-10%", "Decreased less than 5%", "Increased less than 5%", "Increased 5-10%", "Increased 10-20%", "Increased 20-40%", "Increased more than 40%"])

    sim_grants_melted["Population"] = pop_melted["Population"]

    return sim_grants_melted

In [13]:
sim_grants_30 = grants_simulation("./Data/FA Grants Tables - Python.xlsx",
    0.30, 0.30)

sim_grants_20 = grants_simulation("./Data/FA Grants Tables - Python.xlsx",
    0.20, 0.30)

sim_grants_10 = grants_simulation("./Data/FA Grants Tables - Python.xlsx",
    0.10, 0.30)

sim_grants_0 = grants_simulation("./Data/FA Grants Tables - Python.xlsx",
    0.00, 0.30)

In [14]:
sim_grants_30_melted = grants_delta(sim_grants_30)

sim_grants_20_melted = grants_delta(sim_grants_20)

sim_grants_10_melted = grants_delta(sim_grants_10)

sim_grants_0_melted = grants_delta(sim_grants_0)

In [15]:
from datetime import datetime as tm

with pd.ExcelWriter(f"./Data/Output/Simulation Results {tm.today().strftime("%d_%m_%Y")}.xlsx") as writer:
    sim_grants_30_melted.to_excel(writer, sheet_name = "30% Minimum Grant")
    sim_grants_20_melted.to_excel(writer, sheet_name = "20% Minimum Grant")
    sim_grants_10_melted.to_excel(writer, sheet_name = "10% Minimum Grant")
    sim_grants_0_melted.to_excel(writer, sheet_name = "0% Minimum Grant")

In [16]:
def net_present_value(flows):
    return sum(flow / (1 + 0.025) ** i for i, flow in enumerate(flows)) # Parameters show a projected CPI of 2.5

from IPython.display import Markdown, display_markdown


In [17]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.precision', 1)
pd.options.display.float_format = '{:,.1f}'.format

def display_data(data, state):
    if state == "all":
        sub = data
        gby = ["Jurisdiction", "ACLG", "LGA"]
        xtab = "Jurisdiction"
    else:
        sub = data[data["Jurisdiction"] == state]
        gby = ["ACLG", "LGA"]
        xtab = "ACLG"

    output_dict = {}
    
    k = sub.groupby(gby).agg(
        {
            "Expected Change in GPG ($)" : net_present_value
        }
    ).reset_index()

    output_dict["Largest total gains in the 2025-26 to 2028-29 period"] = k.loc[k["Expected Change in GPG ($)"].idxmax()].to_dict()
    output_dict["Largest total losses in the 2025-26 to 2028-29 period"] = k.loc[k["Expected Change in GPG ($)"].idxmin()].to_dict()

    display(pd.DataFrame(output_dict).T)

    display(Markdown("**Distribution of change in GPG entitlement in 2026-27**"))

    k = sub[sub["Year"] == 2026]

    output_df = pd.crosstab(index=k[xtab], columns=k["Expected Change in GPG (%) Bin"])
    
    display(output_df)

    display(Markdown("\\newpage"))


## A 20% Per Capita Minimum Grant

### National

In [19]:
display_data(sim_grants_20_melted, "all")

,Jurisdiction,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,Queensland,RTL,Torres Strait Island Regional Council,"9,344,772.0"
Largest total losses in the 2025-26 to 2028-29 period,Queensland,UCC,Brisbane City Council,"-49,269,604.8"


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased 20-40%,Decreased 10-20%,Decreased 5-10%,Decreased less than 5%,Increased less than 5%,Increased 5-10%,Increased 10-20%
Jurisdiction,,,,,,,
New South Wales,27,5,0,10,14,17,58
Northern Territory,6,1,0,0,0,6,4
Queensland,17,3,3,2,3,1,48
South Australia,17,3,1,3,2,18,23
Tasmania,5,0,1,2,6,10,5
Victoria,15,3,3,7,8,30,13
Western Australia,29,3,3,2,8,15,75


\newpage

### New South Wales

In [20]:
display_data(sim_grants_20_melted, "New South Wales")

,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,RAL,Lachlan Council,"4,609,135.1"
Largest total losses in the 2025-26 to 2028-29 period,UDV,Canterbury-Bankstown Council,"-13,976,297.4"


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased 20-40%,Decreased 10-20%,Decreased less than 5%,Increased less than 5%,Increased 5-10%,Increased 10-20%
ACLG,,,,,,
RAL,0,0,0,0,1,21
RAM,0,0,0,0,0,13
RAS,0,0,0,0,0,1
RAV,0,0,0,0,3,16
RTL,0,0,0,0,0,1
RTM,0,0,0,0,0,1
RTX,0,0,0,0,0,3
UCC,1,0,0,0,0,0
UDL,4,0,0,0,0,0


\newpage

### Victoria

In [21]:
display_data(sim_grants_20_melted, "Victoria")

,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,URM,East Gippsland Shire Council,"4,978,065.7"
Largest total losses in the 2025-26 to 2028-29 period,UCC,Melbourne City Council (City of Melbourne),"-6,663,865.9"


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased 20-40%,Decreased 10-20%,Decreased 5-10%,Decreased less than 5%,Increased less than 5%,Increased 5-10%,Increased 10-20%
ACLG,,,,,,,
RAL,0,0,0,0,0,1,6
RAM,0,0,0,0,0,0,1
RAV,0,0,0,0,0,10,6
UCC,1,0,0,0,0,0,0
UDL,5,1,1,0,0,0,0
UDV,9,1,1,4,0,0,0
UFL,0,0,0,0,1,0,0
UFM,0,0,1,0,1,1,0
UFS,0,0,0,0,0,2,0


\newpage

### Queensland

In [22]:
display_data(sim_grants_20_melted, "Queensland")

,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,RTL,Torres Strait Island Regional Council,"9,344,772.0"
Largest total losses in the 2025-26 to 2028-29 period,UCC,Brisbane City Council,"-49,269,604.8"


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased 20-40%,Decreased 10-20%,Decreased 5-10%,Decreased less than 5%,Increased less than 5%,Increased 5-10%,Increased 10-20%
ACLG,,,,,,,
RAM,0,0,0,0,0,0,1
RAV,0,0,0,0,0,0,8
RTL,0,0,0,0,0,0,8
RTM,0,0,0,0,0,0,15
RTS,0,0,0,0,0,0,8
RTX,0,0,0,0,0,0,7
UCC,1,0,0,0,0,0,0
UFM,1,1,0,0,0,0,0
UFS,0,1,0,0,0,0,0


\newpage

### South Australia

In [23]:
display_data(sim_grants_20_melted, "South Australia")

,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,UFL,City of Playford,"3,889,288.7"
Largest total losses in the 2025-26 to 2028-29 period,UDV,City of Port Adelaide Enfield,"-5,048,686.2"


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased 20-40%,Decreased 10-20%,Decreased 5-10%,Decreased less than 5%,Increased less than 5%,Increased 5-10%,Increased 10-20%
ACLG,,,,,,,
RAL,0,0,0,0,0,6,4
RAM,1,0,0,1,0,1,7
RAS,1,0,0,0,0,0,9
RAV,0,0,0,0,1,4,2
RSG,0,1,0,0,0,0,0
UCC,1,0,0,0,0,0,0
UDL,3,0,0,0,0,0,0
UDM,7,0,0,0,0,0,0
UDS,2,0,0,0,0,0,0


\newpage

### Western Australia

In [24]:
display_data(sim_grants_20_melted, "Western Australia")

,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,RTL,Shire of Derby-West Kimberley,"3,216,259.6"
Largest total losses in the 2025-26 to 2028-29 period,UDV,City of Stirling,"-9,140,445.7"


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased 20-40%,Decreased 10-20%,Decreased 5-10%,Decreased less than 5%,Increased less than 5%,Increased 5-10%,Increased 10-20%
ACLG,,,,,,,
RAL,0,0,1,0,0,5,3
RAM,0,0,0,0,2,3,5
RAS,1,0,0,0,0,1,49
RAV,0,0,0,0,1,2,0
RSG,0,0,1,0,0,0,0
RTL,1,0,0,0,1,2,4
RTM,0,0,0,0,0,1,4
RTS,0,0,0,0,0,0,5
RTX,0,0,0,0,0,0,5


\newpage

### Tasmania

In [25]:
display_data(sim_grants_20_melted, "Tasmania")

,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,URS,Central Coast Council,"933,434.7"
Largest total losses in the 2025-26 to 2028-29 period,UFM,Clarence City Council,"-2,669,712.2"


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased 20-40%,Decreased 5-10%,Decreased less than 5%,Increased less than 5%,Increased 5-10%,Increased 10-20%
ACLG,,,,,,
RAL,0,0,0,0,4,2
RAM,0,1,0,0,2,1
RAS,0,0,0,0,0,2
RAV,0,0,1,3,3,0
UCC,1,0,0,0,0,0
UFM,3,0,0,0,0,0
UFS,0,0,1,1,0,0
URM,1,0,0,0,0,0
URS,0,0,0,2,1,0


\newpage

### Northern Territory

In [26]:
display_data(sim_grants_20_melted, "Northern Territory")

,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,RTL,East Arnhem Regional Council,"1,895,974.5"
Largest total losses in the 2025-26 to 2028-29 period,UCC,City of Darwin,"-3,089,747.2"


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased 20-40%,Decreased 10-20%,Increased 5-10%,Increased 10-20%
ACLG,,,,
RTL,0,0,4,4
RTM,1,0,1,0
RTS,1,0,0,0
RTX,0,0,1,0
UCC,1,0,0,0
UFS,1,0,0,0
URM,1,0,0,0
URS,1,1,0,0


\newpage

## A 10% Per Capita Minimum Grant

### National

In [27]:
display_data(sim_grants_10_melted, "all")

,Jurisdiction,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,Queensland,RTL,Torres Strait Island Regional Council,"18,663,747.3"
Largest total losses in the 2025-26 to 2028-29 period,Queensland,UCC,Brisbane City Council,"-98,539,209.5"


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased more than 40%,Decreased 20-40%,Decreased 10-20%,Decreased 5-10%,Decreased less than 5%,Increased less than 5%,Increased 5-10%,Increased 10-20%,Increased 20-40%
Jurisdiction,,,,,,,,,
New South Wales,27,5,0,5,6,6,7,17,58
Northern Territory,6,1,0,0,0,0,0,6,4
Queensland,17,3,3,1,1,1,2,1,48
South Australia,17,3,1,1,2,0,2,18,23
Tasmania,5,0,1,1,1,1,5,10,5
Victoria,15,3,3,0,7,4,4,30,13
Western Australia,29,3,3,2,0,5,3,15,75


\newpage

### New South Wales

In [28]:
display_data(sim_grants_10_melted, "New South Wales")

,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,RAL,Lachlan Council,"9,252,503.7"
Largest total losses in the 2025-26 to 2028-29 period,UDV,Canterbury-Bankstown Council,"-27,952,594.7"


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased more than 40%,Decreased 20-40%,Decreased 5-10%,Decreased less than 5%,Increased less than 5%,Increased 5-10%,Increased 10-20%,Increased 20-40%
ACLG,,,,,,,,
RAL,0,0,0,0,0,0,1,21
RAM,0,0,0,0,0,0,0,13
RAS,0,0,0,0,0,0,0,1
RAV,0,0,0,0,0,0,3,16
RTL,0,0,0,0,0,0,0,1
RTM,0,0,0,0,0,0,0,1
RTX,0,0,0,0,0,0,0,3
UCC,1,0,0,0,0,0,0,0
UDL,4,0,0,0,0,0,0,0


\newpage

### Victoria

In [29]:
display_data(sim_grants_10_melted, "Victoria")

,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,URM,East Gippsland Shire Council,"10,000,278.6"
Largest total losses in the 2025-26 to 2028-29 period,UCC,Melbourne City Council (City of Melbourne),"-13,327,731.8"


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased more than 40%,Decreased 20-40%,Decreased 10-20%,Decreased less than 5%,Increased less than 5%,Increased 5-10%,Increased 10-20%,Increased 20-40%
ACLG,,,,,,,,
RAL,0,0,0,0,0,0,1,6
RAM,0,0,0,0,0,0,0,1
RAV,0,0,0,0,0,0,10,6
UCC,1,0,0,0,0,0,0,0
UDL,5,1,1,0,0,0,0,0
UDV,9,1,1,4,0,0,0,0
UFL,0,0,0,0,1,0,0,0
UFM,0,0,1,0,1,0,1,0
UFS,0,0,0,0,0,0,2,0


\newpage

### Queensland

In [30]:
display_data(sim_grants_10_melted, "Queensland")

,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,RTL,Torres Strait Island Regional Council,"18,663,747.3"
Largest total losses in the 2025-26 to 2028-29 period,UCC,Brisbane City Council,"-98,539,209.5"


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased more than 40%,Decreased 20-40%,Decreased 10-20%,Decreased 5-10%,Decreased less than 5%,Increased less than 5%,Increased 5-10%,Increased 10-20%,Increased 20-40%
ACLG,,,,,,,,,
RAM,0,0,0,0,0,0,0,0,1
RAV,0,0,0,0,0,0,0,0,8
RTL,0,0,0,0,0,0,0,0,8
RTM,0,0,0,0,0,0,0,0,15
RTS,0,0,0,0,0,0,0,0,8
RTX,0,0,0,0,0,0,0,0,7
UCC,1,0,0,0,0,0,0,0,0
UFM,1,1,0,0,0,0,0,0,0
UFS,0,1,0,0,0,0,0,0,0


\newpage

### South Australia

In [31]:
display_data(sim_grants_10_melted, "South Australia")

,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,UFL,City of Playford,"7,687,683.3"
Largest total losses in the 2025-26 to 2028-29 period,UDV,City of Port Adelaide Enfield,"-10,097,372.4"


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased more than 40%,Decreased 20-40%,Decreased 10-20%,Decreased 5-10%,Decreased less than 5%,Increased 5-10%,Increased 10-20%,Increased 20-40%
ACLG,,,,,,,,
RAL,0,0,0,0,0,0,6,4
RAM,1,0,0,0,1,0,1,7
RAS,1,0,0,0,0,0,0,9
RAV,0,0,0,0,0,1,4,2
RSG,0,1,0,0,0,0,0,0
UCC,1,0,0,0,0,0,0,0
UDL,3,0,0,0,0,0,0,0
UDM,7,0,0,0,0,0,0,0
UDS,2,0,0,0,0,0,0,0


\newpage

### Western Australia

In [32]:
display_data(sim_grants_10_melted, "Western Australia")

,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,RTL,Shire of Derby-West Kimberley,"6,356,727.5"
Largest total losses in the 2025-26 to 2028-29 period,UDV,City of Stirling,"-18,280,891.5"


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased more than 40%,Decreased 20-40%,Decreased 10-20%,Decreased 5-10%,Increased less than 5%,Increased 5-10%,Increased 10-20%,Increased 20-40%
ACLG,,,,,,,,
RAL,0,0,1,0,0,0,5,3
RAM,0,0,0,0,1,1,3,5
RAS,1,0,0,0,0,0,1,49
RAV,0,0,0,0,1,0,2,0
RSG,0,0,1,0,0,0,0,0
RTL,1,0,0,0,1,0,2,4
RTM,0,0,0,0,0,0,1,4
RTS,0,0,0,0,0,0,0,5
RTX,0,0,0,0,0,0,0,5


\newpage

### Tasmania

In [33]:
display_data(sim_grants_10_melted, "Tasmania")

,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,URS,Central Coast Council,"1,879,667.0"
Largest total losses in the 2025-26 to 2028-29 period,UFM,Clarence City Council,"-5,339,424.5"


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased more than 40%,Decreased 10-20%,Decreased 5-10%,Decreased less than 5%,Increased less than 5%,Increased 5-10%,Increased 10-20%,Increased 20-40%
ACLG,,,,,,,,
RAL,0,0,0,0,0,0,4,2
RAM,0,1,0,0,0,0,2,1
RAS,0,0,0,0,0,0,0,2
RAV,0,0,1,0,0,3,3,0
UCC,1,0,0,0,0,0,0,0
UFM,3,0,0,0,0,0,0,0
UFS,0,0,0,1,0,1,0,0
URM,1,0,0,0,0,0,0,0
URS,0,0,0,0,1,1,1,0


\newpage

### Northern Territory

In [34]:
display_data(sim_grants_10_melted, "Northern Territory")

,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,RTL,East Arnhem Regional Council,"3,759,292.4"
Largest total losses in the 2025-26 to 2028-29 period,UCC,City of Darwin,"-6,179,494.5"


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased more than 40%,Decreased 20-40%,Increased 10-20%,Increased 20-40%
ACLG,,,,
RTL,0,0,4,4
RTM,1,0,1,0
RTS,1,0,0,0
RTX,0,0,1,0
UCC,1,0,0,0
UFS,1,0,0,0
URM,1,0,0,0
URS,1,1,0,0


\newpage

## A World With No Per Capita Minimum Grant

### National

In [35]:
display_data(sim_grants_0_melted, "all")

,Jurisdiction,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,Queensland,RTL,Torres Strait Island Regional Council,"28,030,401.0"
Largest total losses in the 2025-26 to 2028-29 period,Queensland,UCC,Brisbane City Council,"-147,808,814.3"


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased more than 40%,Decreased 20-40%,Decreased 10-20%,Decreased 5-10%,Decreased less than 5%,Increased less than 5%,Increased 5-10%,Increased 10-20%,Increased 20-40%,Increased more than 40%
Jurisdiction,,,,,,,,,,
New South Wales,30,2,2,7,2,4,4,9,63,8
Northern Territory,7,0,0,0,0,0,0,0,10,0
Queensland,19,3,2,1,0,1,2,0,12,37
South Australia,19,2,1,0,2,0,1,6,36,0
Tasmania,5,1,0,1,1,1,0,7,13,0
Victoria,18,2,1,3,4,2,2,12,35,0
Western Australia,32,3,1,1,0,3,3,4,48,40


\newpage

### New South Wales

In [36]:
display_data(sim_grants_0_melted, "New South Wales")

,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,RAL,Lachlan Council,"13,919,344.3"
Largest total losses in the 2025-26 to 2028-29 period,UDV,Canterbury-Bankstown Council,"-41,928,892.1"


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased more than 40%,Decreased 20-40%,Decreased 10-20%,Decreased 5-10%,Decreased less than 5%,Increased less than 5%,Increased 5-10%,Increased 10-20%,Increased 20-40%,Increased more than 40%
ACLG,,,,,,,,,,
RAL,0,0,0,0,0,0,0,0,21,1
RAM,0,0,0,0,0,0,0,0,9,4
RAS,0,0,0,0,0,0,0,0,0,1
RAV,0,0,0,0,0,0,0,1,18,0
RTL,0,0,0,0,0,0,0,0,0,1
RTM,0,0,0,0,0,0,0,0,0,1
RTX,0,0,0,0,0,0,0,0,3,0
UCC,1,0,0,0,0,0,0,0,0,0
UDL,4,0,0,0,0,0,0,0,0,0


\newpage

### Victoria

In [37]:
display_data(sim_grants_0_melted, "Victoria")

,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,URM,East Gippsland Shire Council,"14,886,559.7"
Largest total losses in the 2025-26 to 2028-29 period,UCC,Melbourne City Council (City of Melbourne),"-19,991,597.7"


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased more than 40%,Decreased 20-40%,Decreased 10-20%,Decreased 5-10%,Decreased less than 5%,Increased less than 5%,Increased 5-10%,Increased 10-20%,Increased 20-40%
ACLG,,,,,,,,,
RAL,0,0,0,0,0,0,0,0,7
RAM,0,0,0,0,0,0,0,0,1
RAV,0,0,0,0,0,0,0,0,16
UCC,1,0,0,0,0,0,0,0,0
UDL,6,1,0,0,0,0,0,0,0
UDV,10,0,1,2,2,0,0,0,0
UFL,0,0,0,0,0,0,1,0,0
UFM,0,1,0,0,0,0,1,1,0
UFS,0,0,0,0,0,0,0,1,1


\newpage

### Queensland

In [38]:
display_data(sim_grants_0_melted, "Queensland")

,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,RTL,Torres Strait Island Regional Council,"28,030,401.0"
Largest total losses in the 2025-26 to 2028-29 period,UCC,Brisbane City Council,"-147,808,814.3"


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased more than 40%,Decreased 20-40%,Decreased 10-20%,Decreased 5-10%,Increased less than 5%,Increased 5-10%,Increased 20-40%,Increased more than 40%
ACLG,,,,,,,,
RAM,0,0,0,0,0,0,0,1
RAV,0,0,0,0,0,0,8,0
RTL,0,0,0,0,0,0,1,7
RTM,0,0,0,0,0,0,1,14
RTS,0,0,0,0,0,0,0,8
RTX,0,0,0,0,0,0,0,7
UCC,1,0,0,0,0,0,0,0
UFM,2,0,0,0,0,0,0,0
UFS,0,1,0,0,0,0,0,0


\newpage

### South Australia

In [39]:
display_data(sim_grants_0_melted, "South Australia")

,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,UFL,City of Playford,"11,707,984.0"
Largest total losses in the 2025-26 to 2028-29 period,UDV,City of Port Adelaide Enfield,"-15,146,058.6"


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased more than 40%,Decreased 20-40%,Decreased 10-20%,Decreased less than 5%,Increased 5-10%,Increased 10-20%,Increased 20-40%
ACLG,,,,,,,
RAL,0,0,0,0,0,1,9
RAM,1,0,0,1,0,1,7
RAS,1,0,0,0,0,0,9
RAV,0,0,0,0,0,2,5
RSG,1,0,0,0,0,0,0
UCC,1,0,0,0,0,0,0
UDL,3,0,0,0,0,0,0
UDM,7,0,0,0,0,0,0
UDS,2,0,0,0,0,0,0


\newpage

### Western Australia

In [40]:
display_data(sim_grants_0_melted, "Western Australia")

,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,RTL,Shire of Derby-West Kimberley,"9,578,148.3"
Largest total losses in the 2025-26 to 2028-29 period,UDV,City of Stirling,"-27,421,337.2"


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased more than 40%,Decreased 20-40%,Decreased 10-20%,Decreased 5-10%,Increased less than 5%,Increased 5-10%,Increased 10-20%,Increased 20-40%,Increased more than 40%
ACLG,,,,,,,,,
RAL,0,1,0,0,0,0,0,8,0
RAM,0,0,0,0,1,1,0,8,0
RAS,1,0,0,0,0,0,0,23,27
RAV,0,0,0,0,1,0,0,2,0
RSG,0,1,0,0,0,0,0,0,0
RTL,1,0,0,0,0,1,0,6,0
RTM,0,0,0,0,0,0,1,1,3
RTS,0,0,0,0,0,0,0,0,5
RTX,0,0,0,0,0,0,0,0,5


\newpage

### Tasmania

In [41]:
display_data(sim_grants_0_melted, "Tasmania")

,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,URS,Central Coast Council,"2,859,488.8"
Largest total losses in the 2025-26 to 2028-29 period,UFM,Clarence City Council,"-8,009,136.7"


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased more than 40%,Decreased 20-40%,Decreased 5-10%,Decreased less than 5%,Increased less than 5%,Increased 10-20%,Increased 20-40%
ACLG,,,,,,,
RAL,0,0,0,0,0,0,6
RAM,0,1,0,0,0,1,2
RAS,0,0,0,0,0,0,2
RAV,0,0,1,0,0,4,2
UCC,1,0,0,0,0,0,0
UFM,3,0,0,0,0,0,0
UFS,0,0,0,1,0,1,0
URM,1,0,0,0,0,0,0
URS,0,0,0,0,1,1,1


\newpage

### Northern Territory

In [42]:
display_data(sim_grants_0_melted, "Northern Territory")

,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,RTL,East Arnhem Regional Council,"5,669,597.2"
Largest total losses in the 2025-26 to 2028-29 period,UCC,City of Darwin,"-9,269,241.7"


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased more than 40%,Increased 20-40%
ACLG,,
RTL,0,8
RTM,1,1
RTS,1,0
RTX,0,1
UCC,1,0
UFS,1,0
URM,1,0
URS,2,0


\newpage

## And for sanity reasons, status quo

### National

In [43]:
display_data(sim_grants_30_melted, "all")

,Jurisdiction,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,New South Wales,RAL,Berrigan Shire Council,0.0
Largest total losses in the 2025-26 to 2028-29 period,New South Wales,RAL,Berrigan Shire Council,0.0


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased less than 5%
Jurisdiction,
New South Wales,131
Northern Territory,17
Queensland,77
South Australia,67
Tasmania,29
Victoria,79
Western Australia,135


\newpage

### New South Wales

In [44]:
display_data(sim_grants_30_melted, "New South Wales")

,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,RAL,Berrigan Shire Council,0.0
Largest total losses in the 2025-26 to 2028-29 period,RAL,Berrigan Shire Council,0.0


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased less than 5%
ACLG,
RAL,22
RAM,13
RAS,1
RAV,19
RTL,1
RTM,1
RTX,3
UCC,1
UDL,4


\newpage

### Victoria

In [45]:
display_data(sim_grants_30_melted, "Victoria")

,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,RAL,Buloke Shire Council,0.0
Largest total losses in the 2025-26 to 2028-29 period,RAL,Buloke Shire Council,0.0


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased less than 5%
ACLG,
RAL,7
RAM,1
RAV,16
UCC,1
UDL,7
UDV,15
UFL,1
UFM,3
UFS,2


\newpage

### Queensland

In [46]:
display_data(sim_grants_30_melted, "Queensland")

,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,RAM,Balonne Shire Council,0.0
Largest total losses in the 2025-26 to 2028-29 period,RAM,Balonne Shire Council,0.0


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased less than 5%
ACLG,
RAM,1
RAV,8
RTL,8
RTM,15
RTS,8
RTX,7
UCC,1
UFM,2
UFS,1


\newpage

### South Australia

In [47]:
display_data(sim_grants_30_melted, "South Australia")

,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,RAL,Adelaide Plains Council,0.0
Largest total losses in the 2025-26 to 2028-29 period,RAL,Adelaide Plains Council,0.0


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased less than 5%
ACLG,
RAL,10
RAM,10
RAS,10
RAV,7
RSG,1
UCC,1
UDL,3
UDM,7
UDS,2


\newpage

### Western Australia

In [48]:
display_data(sim_grants_30_melted, "Western Australia")

,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,RAL,Shire of Carnarvon,0.0
Largest total losses in the 2025-26 to 2028-29 period,RAL,Shire of Carnarvon,0.0


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased less than 5%
ACLG,
RAL,9
RAM,10
RAS,51
RAV,3
RSG,1
RTL,8
RTM,5
RTS,5
RTX,5


\newpage

### Tasmania

In [49]:
display_data(sim_grants_30_melted, "Tasmania")

,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,RAL,Break O'day Council,0.0
Largest total losses in the 2025-26 to 2028-29 period,RAL,Break O'day Council,0.0


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased less than 5%
ACLG,
RAL,6
RAM,4
RAS,2
RAV,7
UCC,1
UFM,3
UFS,2
URM,1
URS,3


\newpage

### Northern Territory

In [50]:
display_data(sim_grants_30_melted, "Northern Territory")

,ACLG,LGA,Expected Change in GPG ($)
Largest total gains in the 2025-26 to 2028-29 period,RTL,Barkly Regional Council,0.0
Largest total losses in the 2025-26 to 2028-29 period,RTL,Barkly Regional Council,0.0


**Distribution of change in GPG entitlement in 2026-27**

Expected Change in GPG (%) Bin,Decreased less than 5%
ACLG,
RTL,8
RTM,2
RTS,1
RTX,1
UCC,1
UFS,1
URM,1
URS,2


\newpage